# Sales Analysis

#### Importing the necessary libraries

In [1]:
import pandas as pd
import os

#### Listing all the files in current working directory

In [2]:
os.listdir()

['.config', 'sample_data']

In [17]:
files = [file for file in os.listdir()]

In [18]:
print(files)

['.config', '.ipynb_checkpoints', 'Sales_April_2019.csv', 'Sales Data', 'sample_data']


#### Merging 12 months of sales data into single csv file

In [ ]:
data = pd.DataFrame()
for file in files:
  df = pd.read_csv('./' + file)
  data = pd.concat([data, df])

#### Data Cleaning/Preprocessing:

#### Augment data with additional columns

In [ ]:
data.dropna(inplace = True)

In [ ]:
data['Order Month'] = data['Order Date'].str[0:2]
data['Order Month']

#### Dropping null/na values

In [ ]:
temp_df = data[data['Order Date'].str[0:2] == 'Or']
temp_df

#### Resolving descrepancy in Order date column

In [ ]:
data = data[data['Order Date'].str[0:2] != 'Or']
data['Order Month'] = pd.to_numeric(data['Order Month'])

#### Changing column data type to respective numeric datatype

In [ ]:
data['Quantity Ordered'] = pd.to_numeric(data['Quantity Ordered'])
data['Price Each'] = pd.to_numeric(data['Price Each'])

#### Calculating the required column values and augmenting

In [ ]:
data['Sales Amount'] = data['Quantity Ordered'] * data['Price Each']
data['Sales Amount']

### Question 1: What was the best month for sales? What were the profits made that month?

In [ ]:
results = data[['Sales Amount', 'Order Month']].groupby('Order Month').sum()
results

#### Importing visualization libraries, and plotting a bar chart indicating sales amount per month

In [ ]:
import matplotlib.pyplot as plt

months = range(1, 13)

plt.bar(months, results['Sales Amount'])
plt.xticks(months)
plt.ylabel('Sales in USD $')
plt.xlabel('Months')
plt.show()

### Question 2: What city had the highest number of sales?

#### Add city column

In [ ]:
def get_city(address):
    return address.split(',')[1]

def get_state(address):
    return address.split(',')[2].split(' ')[1]

data['City'] = data['Purchase Address'].apply(lambda x: f"{get_city(x)} ({get_state(x)})")

In [ ]:
results = data[['Sales Amount', 'City']].groupby('City').sum()
results

In [ ]:
cities = [city for city, df in data.groupby('City')]

plt.bar(cities, results['Sales Amount'])
plt.xticks(cities, rotation = 45, size = 8)
plt.ylabel('Sales in USD $')
plt.xlabel('Cities')
plt.tight_layout()

### Question 3: What time should we display advertisements to maximize likelihood of customers buying product?

In [ ]:
data['Order Date'] = pd.to_datetime(data['Order Date'])

In [ ]:
data['Hour'] = data['Order Date'].dt.hour

In [ ]:
data['Minute'] = data['Order Date'].dt.minute

In [ ]:
hours = [hour for hour, df in data.groupby('Hour')]

plt.plot(hours, data.groupby('Hour').count())
plt.xticks(hours)
plt.xlabel("Hour of day")
plt.ylabel('Number of Orders')
plt.grid()
plt.show()

### Question 4: What products are most often sold together?

In [ ]:
df = data[data['Order ID'].duplicated(keep = False)]
df['Grouped Products'] = df.groupby('Order ID')['Product'].transform(lambda x: ', '.join(x))

In [ ]:
df = df[['Order ID', 'Grouped Products']].drop_duplicates()

In [ ]:
from itertools import combinations
from collections import Counter

In [ ]:
count = Counter()

for row in df['Grouped Products']:
    row_list = row.split(',')
    count.update(Counter(combinations(row_list, 2)))

count.most_common(10)

In [ ]:
for key, value in count.most_common(10):
    print(key, value)

### Question 5: What product sold the most? Why do you think it sold the most?

In [ ]:
products_grouped = data.drop('Order Date', axis = 1).groupby('Product')

In [ ]:
quantity_ordered = products_grouped.sum()['Quantity Ordered']

In [ ]:
quantity_ordered.sort_values(ascending=False)

In [ ]:
products = [product for product, df in products_grouped]

plt.bar(products, quantity_ordered)
plt.xticks(rotation = 'vertical')
plt.ylabel('Quantity ordered')
plt.xlabel('Product ordered')
plt.show()

In [ ]:
prices = data[['Product', 'Price Each']].groupby('Product').mean()['Price Each']

In [ ]:
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.bar(keys, quantity_ordered, color = 'g')
ax2.plot(keys, prices, color = 'b')

ax1.set_xlabel('Product')
ax1.set_ylabel('Quantity ordered', color = 'g')
ax2.set_ylabel('Price in $', color = 'b')
ax1.set_xticklabels(keys, rotation = 'vertical', size = 8)

fig.show()